In [1]:
import repeated_play

In [2]:
import sympy as sym

import tqdm

In [4]:
import pandas as pd

import itertools

In [27]:
import glob

In [36]:
def seperate_equilibria_and_non(df):
    """A functions that takes a dataframe with points and seperates the
    points are Nash and those that are not.
    
    Returns two dataframes."""
    per_player = df.groupby('ID')['condition A'].all()

    eq_players_ID = per_player[per_player == True].index

    noneq_players_ID = per_player[per_player == False].index
    
    return df[df['ID'].isin(eq_players_ID)], df[df['ID'].isin(noneq_players_ID)]

In [5]:
def transformed_matrix(coplayer, analytical=False):
    
    if analytical == False:
    
        return np.array([[coplayer[0], (1 - coplayer[0]), 0, 0, 0, 0, 0, 0],
                        [0, 0, coplayer[1], (1 - coplayer[1]), 0, 0, 0, 0],
                        [0, 0, 0, 0, coplayer[2], (1 - coplayer[2]), 0, 0],
                        [0, 0, 0, 0, 0, 0, coplayer[3], (1 - coplayer[3])],
                        [coplayer[4], (1 - coplayer[4]), 0, 0, 0, 0, 0, 0],
                        [0, 0, coplayer[5], (1 - coplayer[5]), 0, 0, 0, 0],
                        [0, 0, 0, 0, coplayer[6], (1 - coplayer[6]), 0, 0],
                        [0, 0, 0, 0, 0, 0, coplayer[7], (1 - coplayer[7])]])
    
    if analytical == True:

        return sym.Matrix([[coplayer[0], (1 - coplayer[0]), 0, 0, 0, 0, 0, 0],
                        [0, 0, coplayer[1], (1 - coplayer[1]), 0, 0, 0, 0],
                        [0, 0, 0, 0, coplayer[2], (1 - coplayer[2]), 0, 0],
                        [0, 0, 0, 0, 0, 0, coplayer[3], (1 - coplayer[3])],
                        [coplayer[4], (1 - coplayer[4]), 0, 0, 0, 0, 0, 0],
                        [0, 0, coplayer[5], (1 - coplayer[5]), 0, 0, 0, 0],
                        [0, 0, 0, 0, coplayer[6], (1 - coplayer[6]), 0, 0],
                        [0, 0, 0, 0, 0, 0, coplayer[7], (1 - coplayer[7])]])

In [6]:
qs = sym.symbols("q1:9")
ps = sym.symbols("p1:9")

b, c = sym.symbols("b, c")

In [28]:
files = glob.glob("../data/three_bit_against_self_reactive_three/*.csv")

In [29]:
names = ["ID"] + [f"p_{i}" for i in range(1, 9)] + ['label', 'Sp', 'Sq', 'condition A', 'condition B', "b", "c"]

In [30]:
dfs = []

for file in tqdm.tqdm(files):
    
    dfs.append(pd.read_csv(file, names=names))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5000/5000 [00:15<00:00, 325.60it/s]


In [31]:
df = pd.concat(dfs)

In [32]:
df = df.reset_index(drop=True)

In [ ]:
df.columns = names

In [37]:
eq, noneq = seperate_equilibria_and_non(df)

In [15]:
import numpy as np

In [16]:
pure_self_reactive = list(itertools.product([0, 1], repeat=8))

In [131]:
payoffs = []

indices = []

for i, coplayer in tqdm.tqdm(enumerate(pure_self_reactive)):
    
    M = transformed_matrix(coplayer)
    
    stationaries = repeated_play.stationary_distribution(M)
    
    for ss in stationaries:
        
        rho_q = ss[0] + ss[1] + ss[4] + ss[5]
        
        rho_p = sum([ss[i] * ps[i] for i in range(8)])
        
        
        expression =  b * rho_p - c * rho_q
        
        expression = sym.nsimplify(expression.subs({ps[0]: 1}))
        
        payoffs.append(expression)
        
        indices.append(i)
    

256it [00:03, 76.89it/s]


In [220]:
IDs = noneq['ID'].unique()

In [221]:
# set(IDs).difference(noneq[(noneq['label'] == f'N0') & (noneq['condition A'] == False)]['ID'].unique())

In [392]:
strategies = [[indices[i] for i, p in enumerate(payoffs) if p == unique_p] for unique_p in unique_payoffs]

In [256]:
A = set(strategies[0])

B = set([item for sublist in strategies[1:] for item in sublist])

A.difference(B)

{0, 2, 8, 10, 32, 34, 36, 38, 40, 42, 44, 46, 64, 66, 72, 74}

In [257]:
A = set(strategies[1])

B = set([item for sublist in strategies[2:] for item in sublist])

A.difference(B)

{3, 35, 39}

In [269]:
unique_strategies_per_payoff = []

for i, s in enumerate(strategies):

    A = set(s)
    
    check = strategies[:i] + strategies[i + 1:]

    B = set([item for sublist in check for item in sublist])

    unique_strategies_per_payoff.append(list(A.difference(B)))

In [292]:
unique_strategies_per_payoff

[[0, 32, 2, 34, 36, 64, 38, 66, 8, 40, 10, 42, 44, 72, 46, 74],
 [3, 35, 39],
 [255,
  139,
  171,
  173,
  187,
  175,
  203,
  219,
  235,
  237,
  239,
  251,
  253,
  155,
  189,
  191],
 [91, 61],
 [51, 19, 55],
 [27, 59, 63],
 [75, 45],
 [83, 53],
 [67, 37],
 [11, 43, 47],
 [65, 1, 33, 41, 9, 73],
 [89, 81, 17, 49, 57, 25],
 [107, 109, 111, 123, 125, 127],
 [69, 5, 71, 13, 77, 79, 93, 21, 85, 87, 29, 95],
 [99, 101, 103, 115, 117, 119]]

In [310]:
per_payoff_set = []

for i, s in enumerate(unique_strategies_per_payoff):
    
    per_s = []
    
    for j in s:
        
        A = set(noneq[(noneq['label'] == f'N{j}') & (noneq['condition A'] == False)]['ID'].unique())

        to_check = unique_strategies_per_payoff[:i] + unique_strategies_per_payoff[i + 1:]

        B =  [noneq[(noneq['label'] == f'N{l[0]}') & (noneq['condition A'] == False)]['ID'].unique()
              for l in to_check]

        C = set([item for sublist in B for item in sublist])

        per_s += list(A.difference(C))
        
    per_payoff_set.per_s

In [311]:
[i for i, s in enumerate(per_payoff_set) if len(s) == 0]

[1, 2, 3, 6, 7, 8, 9, 10, 12, 14]

In [312]:
[i for i, s in enumerate(per_payoff_set) if len(s) != 0]

[0, 4, 5, 11, 13]

In [375]:
unique_strategies_per_payoff[4]

[51, 19, 55]

In [373]:
unique_payoffs[0]

b*p8

In [382]:
explained = []

for i in unique_strategies_per_payoff[14]:
    
    explained += list(noneq[(noneq['label'] == f'N{i}') & (noneq['condition A'] == False)]['ID'].unique())
    

In [389]:
A = [noneq[(noneq['label'] == f'N{i}') & (noneq['condition A'] == False)]['ID'].unique()
     for i in unique_strategies_per_payoff[0]]

B = set(noneq[(noneq['label'] == f'N55') & (noneq['condition A'] == False)]['ID'].unique())

C = set(noneq[(noneq['label'] == f'N17') & (noneq['condition A'] == False)]['ID'].unique())

D =  set(noneq[(noneq['label'] == f'N5') & (noneq['condition A'] == False)]['ID'].unique())

In [390]:
E = A.union(B).union(C).union(D).union(set(explained))

In [391]:
len(E)

3008

In [395]:
strategies[1]

[3, 7, 35, 39, 131, 135, 163, 167]

In [398]:
A = set(noneq[(noneq['label'] == f'N7') & (noneq['condition A'] == False)]['ID'].unique())

len(A)

542

In [306]:
i = 14

j = unique_strategies_per_payoff[i][0]

A = set(noneq[(noneq['label'] == f'N{j}') & (noneq['condition A'] == False)]['ID'].unique())

to_check = unique_strategies_per_payoff[:i] + unique_strategies_per_payoff[i + 1:]

B =  [noneq[(noneq['label'] == f'N{l[0]}') & (noneq['condition A'] == False)]['ID'].unique()
      for l in to_check]

C = set([item for sublist in B for item in sublist])

In [308]:
A.difference(C)

set()

In [290]:
sym.solve(unique_payoffs[0] - c)[0]

{b: c/p8}

In [323]:
unique_payoffs[4] - (b - c)

b*(p2/4 + p4/4 + p5/4 + p7/4) - b + c/2

In [324]:
unique_payoffs[5] - (b - c)

b*(p2/5 + p4/5 + p5/5 + p7/5 + 1/5) - b + 2*c/5

In [294]:
unique_payoffs[11]

b*(p4/3 + p6/3 + p7/3) - c/3

In [287]:
unique_payoffs[13]

b*(p3/2 + p6/2) - c/2

In [364]:
unique_payoffs[14]

b*(p2/3 + p3/3 + p5/3) - 2*c/3

In [ ]:
sets_ = [noneq[(noneq['label'] == f'N{i}') & (noneq['condition A'] == False)]['ID'].unique() for i in [0,
                                                                                                       18,
                                                                                                       16,
                                                                                                       94]]

In [137]:
I = [[indices[i] for i, p in enumerate(payoffs) if p == unique_payoffs[j]][0] for j in unique_indices]

In [176]:
noneq[(noneq['label'] == f'N0') & (noneq['condition A'] == False)]['ID'].unique()

array([1337, 1451, 2164, ..., 1448, 2627, 3539])

In [172]:
A = set()

for i in sets_:
    A = A.union(set(i))

In [173]:
len(A)

2878

In [116]:
df['ID'].nunique()

5000

In [117]:
sets_ = [noneq[(noneq['label'] == f'N{i}') & (noneq['condition A'] == False)]['ID'].unique() for i in I]

In [ ]:
[noneq[(noneq['label'] == f'N{i}') & (noneq['condition A'] == False)]['ID'].unique()

In [118]:
len(sets_)

6

In [123]:
A = set()

for i in sets_:
    A = A.union(set(i))

In [125]:
len(A)

2765

In [126]:
sets_

[array([1337, 1451, 2164, ..., 1448, 2627, 3539]),
 array([1337, 1451, 2164, ..., 1448, 2627, 3539]),
 array([2825, 1486, 3291,  402, 3736, 1109, 1848, 1874, 3124, 3859, 4688,
        1914, 1243, 2004, 2789,  577, 2005, 1518, 2987, 4879,  952, 3643,
        1861, 1685, 1849, 4702, 2367, 1108, 1646,  371, 4528, 1446,  818,
        2417, 3709,  830, 4072, 1687,  603, 1863, 3899, 4112,  777, 3682,
         987, 1526, 2761, 2946, 4305, 2774, 2760, 2748, 2984, 1057, 4073,
        2358,  366, 3250, 4275, 2189, 3283,  606,  955, 4117, 4103, 2228,
        1721, 2764, 1279, 3335, 4658, 1046,  613,  149, 3902, 3719, 1468,
        4262, 1454, 1642,  639, 4100, 4666, 2203,  765, 4699, 1093,    5,
        4470, 3451, 1246, 1253, 3450, 4471, 4840,  994, 1904,  943, 4667,
        1858,  162, 3929, 4908, 1643, 1125, 3732,  823, 2404,  374,  348,
        1333, 2885, 1426,  449, 4990, 4953,  918, 3806, 1220, 1585,  267,
        2099, 1209,  299, 3608, 1779, 2270, 4629, 1180, 3783,  110, 1619,
         8